In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable 
import torchvision
from torchvision import transforms
from torch import utils
from sklearn.preprocessing import StandardScaler
import random

In [15]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
use_cuda

False

In [16]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [0]:
from torch.utils import data

class Dataset(data.Dataset):
	'Characterizes a dataset for PyTorch'
	def __init__(self, features, labels):
		'Initialization'
		self.labels = labels
		self.features = features
    
	def __len__(self):
		'Denotes the total number of samples'
		return len(self.features)

	def __getitem__(self, index):
		'Generates one sample of data'
		x = self.features[index]
		y = self.labels[index]
		return x, y

In [0]:
train_data_file = "/content/gdrive/My Drive/PITE/data.csv"
data = pd.read_csv(train_data_file)
features = data.drop(["is_true_seed","has_MCParticle",
                      "is_downstream_reconstructible",
                      "has_MCParticle_not_electron",
                     "is_downstream_reconstructible_not_electron",
                     "seed_mva_value", "seed_p",
                     "seed_nbIT", "seed_tx", "seed_ty"] , axis=1).astype(np.float64)
targets = torch.FloatTensor(data.is_downstream_reconstructible.values)
featuresnp = features.drop(features.columns[0], axis=1).values

In [0]:
class LogisticRegressionModel(torch.nn.Module): 
  
    def __init__(self, input_dim): 
        super(LogisticRegressionModel, self).__init__()
        self.linear1 = nn.Linear(input_dim, 8)
        self.elu1 = nn.ELU()
        self.linear2 = nn.Linear(8, 7)
        self.elu2 = nn.ELU()
        self.linear3 = nn.Linear(7, 1)
    def forward(self, x): 
        y = self.linear1(x)
        y = self.elu1(y)
        y = self.linear2(y)
        y = self.elu2(y)
        y = torch.sigmoid(self.linear3(y))
        return y

In [20]:
scaler = StandardScaler()
scaler.fit(featuresnp)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [21]:
features = torch.FloatTensor(scaler.transform(featuresnp))
print(features)

tensor([[-0.3324, -0.0101,  1.9428,  0.7196, -2.0001, -0.2653],
        [-1.1899, -0.0060,  1.3008,  0.7196, -0.5555, -0.5355],
        [ 0.8119, -0.0112,  1.3008,  0.7196, -1.1760, -0.5088],
        ...,
        [ 2.8416, -0.0126, -0.6252, -1.2247, -0.6076, -0.6010],
        [ 3.9854, -0.0120, -0.8392, -1.2247, -1.0459,  0.4157],
        [ 1.6221, -0.0092,  0.0168, -2.1969,  0.7254,  2.2229]])


In [0]:
batch_size = 200
epochs = 500
n_iters = epochs * (len(targets) / batch_size)
input_dim = 6
output_dim = 1
lr_rate = 0.0001

In [0]:
model = LogisticRegressionModel(input_dim).to(device)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [0]:
training_set = Dataset(features, targets)
training_generator = utils.data.DataLoader(training_set, batch_size = batch_size, shuffle = True)

In [25]:
loss = []
accuracy = []

for epoch in range(epochs):
    for features, targets in training_generator:
        features, targets = features.to(device), targets.to(device)
        optimizer.zero_grad()
        pred = model(features).view(200)
        loss = criterion(pred, targets)
        loss.backward()
        optimizer.step()
    print('epoch {}, loss {}'.format(epoch, loss.data))
    if epoch % 5 == 0:
        torch.save(model.state_dict(), "/content/gdrive/My Drive/PITE/model")
        n = 0
        k = random.randint(0, len(training_set))
        for i in range(k-1000, k):
            features, target = training_set[int(i)]
            pred = model(features.to(device))
            pred = 1 if pred > 0.5 else 0
            if pred == target:
              n+=1
        print("accuracy = {}".format(n/1000))
        accuracy.append(n/1000)

epoch 0, loss 0.680752158164978
accuracy = 0.567
epoch 1, loss 0.6827461123466492
epoch 2, loss 0.6766551733016968
epoch 3, loss 0.670265257358551
epoch 4, loss 0.6792043447494507
epoch 5, loss 0.6745888590812683
accuracy = 0.617
epoch 6, loss 0.6636698842048645
epoch 7, loss 0.6597880721092224
epoch 8, loss 0.6634639501571655
epoch 9, loss 0.6663033366203308
epoch 10, loss 0.6580162644386292
accuracy = 0.635
epoch 11, loss 0.6500698924064636


KeyboardInterrupt: ignored